In [1]:
%pip install transformers[torch] huggingface_hub datasets evaluate ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 213.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 560.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 381.2 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Torchtext není k dispozici pro poslední verzi pytorch, budeme tedy využuívat něco jiného ...


In [3]:
from transformers import Trainer, BertForSequenceClassification, BertTokenizer
from datasets import load_from_disk

import torch
import base

In [4]:
base.reset_seed()

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [6]:
train = load_from_disk('./data/sst2/train-logits')
eval = load_from_disk('./data/sst2/eval-logits')
test = load_from_disk('./data/sst2/test-logits')

train_aug = load_from_disk('./data/sst2/train-logits-augmented')

In [7]:
tokenizer = BertTokenizer.from_pretrained("gchhablani/bert-base-cased-finetuned-sst2")

In [8]:
train = train.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the train dataset")
eval = eval.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the eval dataset")
test = test.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the test dataset")

train_aug = train_aug.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the augmented dataset")

In [9]:
base.reset_seed()

In [10]:
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = base.get_training_args(output_dir="./results/bert-base", logging_dir="./logs/bert-base", batch_size=128, epochs=10)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.633700,0.549533,0.719037,0.719241,0.719258,0.719036
2,0.465300,0.484065,0.774083,0.774053,0.773901,0.773951
3,0.376200,0.477339,0.784404,0.786395,0.783573,0.783634
4,0.327300,0.489337,0.788991,0.791029,0.788162,0.788238
5,0.294100,0.468830,0.801606,0.801540,0.801602,0.801561
6,0.276500,0.473168,0.801606,0.801566,0.801476,0.801511
7,0.261300,0.492721,0.795872,0.797331,0.795171,0.795302
8,0.253100,0.480232,0.808486,0.808564,0.808653,0.808480
9,0.244500,0.491007,0.806193,0.806134,0.806106,0.806119
10,0.240600,0.494819,0.806193,0.806301,0.805938,0.806033


TrainOutput(global_step=4210, training_loss=0.3372740793114886, metrics={'train_runtime': 318.3798, 'train_samples_per_second': 1692.287, 'train_steps_per_second': 13.223, 'total_flos': 401089284540000.0, 'train_loss': 0.3372740793114886, 'epoch': 10.0})

In [14]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [15]:
trainer.evaluate(test)

{'eval_loss': 0.27465716004371643,
 'eval_accuracy': 0.8951744617668894,
 'eval_precision': 0.8931336127022499,
 'eval_recall': 0.8953544607934005,
 'eval_f1': 0.8940722936529324,
 'eval_runtime': 8.1757,
 'eval_samples_per_second': 1647.567,
 'eval_steps_per_second': 12.965,
 'epoch': 10.0}

In [16]:
torch.save(model, './models/sst2/bert.pth')

In [17]:
base.reset_seed()

In [18]:
student_model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = base.get_training_args(output_dir="./results/bertTestDistil", logging_dir="./logs/bertTestDistil", remove_unused_columns=False, batch_size=128, epochs=10, temp=5, lambda_param=.5)

In [20]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.118500,2.398569,0.704128,0.707405,0.705165,0.703565
2,2.328300,1.989210,0.759174,0.760040,0.758546,0.758614
3,1.778600,1.836033,0.779817,0.780384,0.779321,0.779441
4,1.469400,1.852941,0.779817,0.784957,0.778522,0.778219
5,1.265300,1.712456,0.786697,0.787427,0.786162,0.786291
6,1.149600,1.704170,0.797018,0.797054,0.796803,0.796877
7,1.073600,1.813394,0.783257,0.786962,0.782152,0.782046
8,1.023000,1.703360,0.801606,0.801831,0.801854,0.801605
9,0.981900,1.723154,0.800459,0.800392,0.800392,0.800392
10,0.962100,1.736029,0.799312,0.799481,0.799013,0.799119


TrainOutput(global_step=4210, training_loss=1.5150246663218156, metrics={'train_runtime': 382.8306, 'train_samples_per_second': 1407.385, 'train_steps_per_second': 10.997, 'total_flos': 401089284540000.0, 'train_loss': 1.5150246663218156, 'epoch': 10.0})

In [22]:
student_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [23]:
trainer.evaluate(test)

{'eval_loss': 1.0660582780838013,
 'eval_accuracy': 0.8928730512249443,
 'eval_precision': 0.8908336252121414,
 'eval_recall': 0.8929261081128524,
 'eval_f1': 0.8917283847291158,
 'eval_runtime': 9.0177,
 'eval_samples_per_second': 1493.731,
 'eval_steps_per_second': 11.755,
 'epoch': 10.0}

In [24]:
torch.save(student_model, './models/sst2/bert-distil.pth')

In [25]:
base.reset_seed()

In [26]:
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
training_args = base.get_training_args(output_dir="./results/bert-base", logging_dir="./logs/bert-base", batch_size=128, epochs=10)

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
)

In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.388000,0.413457,0.827982,0.827978,0.827840,0.827891
2,0.255700,0.423607,0.825688,0.825688,0.825798,0.825673
3,0.220900,0.428542,0.829128,0.829834,0.829555,0.829117
4,0.195800,0.457460,0.826835,0.827262,0.826461,0.826616
5,0.175800,0.499291,0.815367,0.816155,0.814863,0.815034
6,0.161500,0.522447,0.813073,0.813192,0.812821,0.812920
7,0.149700,0.541583,0.819954,0.819935,0.820041,0.819935
8,0.141000,0.567630,0.813073,0.813078,0.812905,0.812965
9,0.135100,0.574606,0.810780,0.811162,0.810400,0.810540
10,0.131200,0.581769,0.813073,0.813078,0.812905,0.812965


TrainOutput(global_step=39660, training_loss=0.1954774442010015, metrics={'train_runtime': 3540.9658, 'train_samples_per_second': 1433.547, 'train_steps_per_second': 11.2, 'total_flos': 3778810595640000.0, 'train_loss': 0.1954774442010015, 'epoch': 10.0})

In [30]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [31]:
trainer.evaluate(test)

{'eval_loss': 0.21611741185188293,
 'eval_accuracy': 0.9204157386785449,
 'eval_precision': 0.9185639385834699,
 'eval_recall': 0.9209728140334594,
 'eval_f1': 0.9195838809560937,
 'eval_runtime': 8.8313,
 'eval_samples_per_second': 1525.249,
 'eval_steps_per_second': 12.003,
 'epoch': 10.0}

In [32]:
torch.save(model, './models/sst2/bert-base-aug.pth')

In [33]:
base.reset_seed()

In [34]:
student_model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
training_args = base.get_training_args(output_dir="./results/bertTestDistil", logging_dir="./logs/bertTestDistil", remove_unused_columns=False, batch_size=128, epochs=10, temp=5, lambda_param=.5)

In [36]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
)

In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.448700,1.315928,0.827982,0.827924,0.827924,0.827924
2,0.802500,1.284763,0.826835,0.826773,0.826840,0.826796
3,0.673200,1.266073,0.833716,0.833668,0.833639,0.833652
4,0.586400,1.306508,0.823394,0.823655,0.823082,0.823212
5,0.521000,1.358838,0.822248,0.824499,0.821451,0.821638
6,0.475400,1.414337,0.818807,0.819536,0.818325,0.818498
7,0.440800,1.431028,0.819954,0.820162,0.819662,0.819781
8,0.414300,1.457472,0.823394,0.824142,0.822914,0.823093
9,0.400400,1.474662,0.814220,0.815245,0.813652,0.813828
10,0.387400,1.483200,0.816514,0.817094,0.816073,0.816234


TrainOutput(global_step=39660, training_loss=0.6149963132672639, metrics={'train_runtime': 3097.5646, 'train_samples_per_second': 1638.752, 'train_steps_per_second': 12.804, 'total_flos': 3778810595640000.0, 'train_loss': 0.6149963132672639, 'epoch': 10.0})

In [38]:
student_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [39]:
trainer.evaluate(test)

{'eval_loss': 0.6758776903152466,
 'eval_accuracy': 0.9255382331106162,
 'eval_precision': 0.924193908662698,
 'eval_recall': 0.9250594547219044,
 'eval_f1': 0.924607041195131,
 'eval_runtime': 9.1618,
 'eval_samples_per_second': 1470.237,
 'eval_steps_per_second': 11.57,
 'epoch': 10.0}

In [40]:
torch.save(student_model, './models/sst2/bert-distil-aug.pth')